In [21]:
%load_ext autoreload
%autoreload 2

import json
import os
from pathlib import Path

config = json.load(open(os.path.expanduser("~/.thesis.conf")))
db_folder = Path(config['datasets']) / 'hisdb'
modules   = Path(config['project']) / 'src'
figures   = Path(config['project']) / 'doc' / 'figures'
import sys
sys.path.append(str(modules))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from datasets.divahisdb import HisDBDataset

In [34]:
dataset = HisDBDataset(db_folder)
samples = [dataset[2], dataset[22], dataset[42]]
dataset.paths

['/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0084r_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0058v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0059v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0008v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0011r_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0072v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0087r_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0067v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0044r_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-codices_fmb-cb-0055_0031v_max.jpg',
 '/media/jakob/bigdata/datasets/hisdb/CB55/img/training/e-co

In [36]:
scale = 0.2
xpos, ypos, size = 600,1200, 600 
box = (xpos,ypos,xpos+size,ypos+size)
for idx, img in enumerate(samples):
    fig = img.resize((int(img.width * scale), int(img.height * scale)))
    name = figures / 'datasets' / ('HisDBSample' + str(idx) + '.jpeg')
    cropped = figures / 'datasets' / ('HisDBSampleBox' + str(idx) + '.jpeg')
    fig.save(name.open('wb'))
    img.crop(box).save(cropped.open('wb'))
    print(name, cropped)

/home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSample0.jpeg /home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSampleBox0.jpeg
/home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSample1.jpeg /home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSampleBox1.jpeg
/home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSample2.jpeg /home/jakob/htw/imi-master-thesis/doc/figures/datasets/HisDBSampleBox2.jpeg
